#                                         Lending Club Loan Data

## Import Data

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


# Pandas options
pd.set_option('display.max_colwidth', 1000, 'display.max_rows', None, 'display.max_columns', None)

# Plotting options
%matplotlib inline
mpl.style.use('ggplot')
sns.set(style='whitegrid')

In [ ]:
SAMPLE = True

In [ ]:
loans = pd.read_csv('../project_1/accepted_2007_to_2018Q4.csv.gz', compression='gzip', low_memory=True) #Load data into pandas

if SAMPLE:
    loans = loans.sample(frac=0.01)

In [ ]:
loans.info()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format 

In [ ]:
loans.head(5)

In [ ]:
loans.shape

## Target Variable

The Target Variable in this loan data is "Loan_Status", which we focus more on either loan status is "Fully Paid" or Charged Off. Will check on value counts of "Loan_Status" variable.

In [ ]:
loans['loan_status'].value_counts(dropna=False) 

Will exclude the loans which status are in Current,Late (31-120 days),In Grace Period,Late (16-30 days),Does not meet the credit policy. Status:Fully Paid,Does not meet the credit policy. Status:Charged Off,Default and missing status

In [ ]:
loans = loans.loc[loans['loan_status'].isin(['Fully Paid', 'Charged Off'])] #Ignore other loan_status

In [ ]:
loans['loan_status']

Loan_Status counts as percentages

In [ ]:
loans['loan_status'].value_counts(normalize=True, dropna=False)

80% are Fully Paid, 20% are Charged Off 

## Data Preparation

Drop columns with >30% data missing

Calculate the missing data percentages of each feature

In [ ]:
missing_data = loans.isnull().mean().sort_values(ascending=False) #calculate % of missing data

In [ ]:
drop_list = sorted(list(missing_data[missing_data > 0.3].index))
print(drop_list)

In [ ]:
len(drop_list) #Missing data >30% columns

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True) # Drop columns

### Keep variables needed for Investors

In [ ]:
keep_list = ['addr_state', 'annual_inc', 'application_type', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'id', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 'verification_status', 'zip_code']

In [ ]:
len(keep_list)

In [ ]:
drop_list = [col for col in loans.columns if col not in keep_list]
print(drop_list) #List remaining features not needed for investors

In [ ]:
len(drop_list)

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True) #Drop Features

In [ ]:
loans.shape #Remaining Columns

## Exploratory Data Analysis

Check dtypes of columns 

In [ ]:
loans.dtypes

In [ ]:
loans.select_dtypes(include=np.object).columns.tolist() # Object data type columns

In [ ]:
loans.select_dtypes(include=np.float).columns.tolist() # Float data type columns

### Analyzing Categorical data

id', 'term', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status',
'issue_d', 'loan_status', 'purpose', 'title',  'zip_code', 'addr_state',  'earliest_cr_line', 'initial_list_status',
'application_type'

#### Id

In [ ]:
loans['id'].describe() #Usually ID's are unique and independent keys

In [ ]:
loans.drop('id', axis=1, inplace=True) #drop id

#### Term

In [ ]:
loans['term'].value_counts(dropna=False)

In [ ]:
loans['term'] = loans['term'].apply(lambda s: np.int8(s.split()[0])) #convert them into numericals

In [ ]:
loans.groupby('term')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

Charge off rates for 60 months are 2x higher than for 36 months term

#### Grade and Sub Grade

In [ ]:
print(sorted(loans['grade'].unique()))

In [ ]:
print(sorted(loans['sub_grade'].unique()))

Both are same, drop one column either grade or sub grade, I am dropping grade column here

In [ ]:
loans.drop('grade', axis=1, inplace=True) #Drop grade

In [ ]:
def plot_var(col_name, full_name, continuous):  # plot function
    """
    Visualize a variable with and without faceting on the loan status.
    - col_name is the variable name in the dataframe
    - full_name is the full variable name
    - continuous is True if the variable is continuous, False otherwise
    """
    f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,3), dpi=90)
    
    # Plot without loan status
    if continuous:
        sns.distplot(loans.loc[loans[col_name].notnull(), col_name], kde=False, ax=ax1)
    else:
        sns.countplot(loans[col_name], order=sorted(loans[col_name].unique()), color='#5975A4', saturation=1, ax=ax1)
    ax1.set_xlabel(full_name)
    ax1.set_ylabel('Count')
    ax1.set_title(full_name)

    # Plot with loan status
    if continuous:
        sns.boxplot(x=col_name, y='loan_status', data=loans, ax=ax2)
        ax2.set_ylabel('')
        ax2.set_title(full_name + ' by Loan Status')
    else:
        charge_off_rates = loans.groupby(col_name)['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']
        sns.barplot(x=charge_off_rates.index, y=charge_off_rates.values, color='#5975A4', saturation=1, ax=ax2)
        ax2.set_ylabel('Fraction of Loans Charged-off')
        ax2.set_title('Charge-off Rate by ' + full_name)
    ax2.set_xlabel(full_name)
    
    plt.tight_layout()

In [ ]:
plot_var('sub_grade', 'Subgrade', continuous=False)

From the above plot, we can find that charge off rates increases with sub grade

#### Employment title

In [ ]:
loans['emp_title'].describe()

In [ ]:
loans.drop(labels='emp_title', axis=1, inplace=True) #too many unique values

#### Employment length

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

Remove years from emp_length variables and convert them into float

In [ ]:
loans['emp_length'].replace('< 1 year', '0 years', inplace=True) #Replace 0 years with 1 year

In [ ]:
loans['emp_length'].replace(to_replace='10+ years', value='10 years', inplace=True) # Replace 10+years with 10 years

In [ ]:
def emp_length_to_int(s):
    if pd.isnull(s): 
        return s
    else:
        return np.int8(s.split()[0])

In [ ]:
loans['emp_length'] = loans['emp_length'].apply(emp_length_to_int)

In [ ]:
loans['emp_length'] = loans['emp_length'].fillna(loans.emp_length.median()) # fill missing values by taking median imputation

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

In [ ]:
loans.groupby('emp_length')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

Charge off rates are decreasing when employment length increases

#### Home Ownership

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

We can merge None, Any as Other

In [ ]:
loans['home_ownership'].replace(['NONE', 'ANY'], 'OTHER', inplace=True)

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

In [ ]:
home_ownership_xt = pd.crosstab(loans['home_ownership'], loans['loan_status'])
home_ownership_xt #Cross table

In [ ]:
home_ownership_xt_pct = home_ownership_xt.div(home_ownership_xt.sum(1).astype(float), axis=0)

home_ownership_xt_pct.plot(kind='bar', 
                   stacked=True, 
                   title='Loan status by Home Ownership')
plt.xlabel('Home Ownership')
plt.ylabel('Loan Status')

Charge off rates are higher for Rent and Own home owner ship with 23% and 21% respectively

In [ ]:
loans.groupby('home_ownership')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

#### Verification Status

In [ ]:
loans.groupby('verification_status')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

Charge off rates are higher for Verified compared to other two types

#### Purpose

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

In [ ]:
loans.groupby('purpose')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

#### Title

In [ ]:
loans['title'].describe()

In [ ]:
loans['title'].value_counts().head

Purpose and Title are quite similar so we can drop either one column

In [ ]:
loans.drop('title', axis=1, inplace=True)

#### Zip code and Address state 

In [ ]:
loans['zip_code'].describe()

In [ ]:
loans['addr_state'].sample(5)

In [ ]:
loans['addr_state'].nunique()

Zip code and address are quite similar which tells about address of applicant, so we will drop one column with more unique values

In [ ]:
loans.groupby('addr_state')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values()

MS state having highest charge off rates with 26%

In [ ]:
loans.drop(labels='zip_code', axis=1, inplace=True)

#### Initial list status

In [ ]:
loans['initial_list_status'].describe()

In [ ]:
plot_var('initial_list_status', 'Initial List Status', continuous=False)

Charge off rates for "w" and "f" status are similar with 20%

#### Application type

In [ ]:
loans['application_type'].value_counts(normalize=True)

In [ ]:
loans.groupby('application_type')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

2% Loan applicants are Joint application types which are having 25% of charge off rates 

### Analyzing Numerical Data

#### Loan amount

In [ ]:
loans['loan_amnt'].describe()

In [ ]:
loans['loan_amnt'] = loans['loan_amnt'].apply(lambda x: np.log10(x+1)) #applying log10 to calculate numerical value

In [ ]:
loans.groupby('loan_status')['loan_amnt'].describe()

#### Interest Rate

In [ ]:
loans['int_rate'].describe()

In [ ]:
loans.groupby('loan_status')['int_rate'].describe()

In [ ]:
loans.groupby('purpose')['int_rate'].describe()

In [ ]:
loans.groupby('sub_grade')['int_rate'].describe()

#### Installment

In [ ]:
loans['installment'].describe()

In [ ]:
loans.groupby('loan_status')['installment'].describe()

#### Annual Income

In [ ]:
loans['annual_inc'].describe()

In [ ]:
loans['log_annual_inc'] = loans['annual_inc'].apply(lambda x: np.log10(x+1))

In [ ]:
loans.drop('annual_inc', axis=1, inplace=True)

In [ ]:
loans['log_annual_inc'].describe()

In [ ]:
loans.groupby('loan_status')['log_annual_inc'].describe()

#### Debt to Income ratio

In [ ]:
loans['dti'].describe()

In [ ]:
loans.groupby('loan_status')['dti'].describe()

#### FICO range

In [ ]:
loans[['fico_range_low', 'fico_range_high']].describe()

In [ ]:
loans[['fico_range_low','fico_range_high']].corr() #Calculating correlation

In [ ]:
loans['fico_score'] = 0.5*loans['fico_range_low'] + 0.5*loans['fico_range_high']

In [ ]:
loans.drop(['fico_range_high', 'fico_range_low'], axis=1, inplace=True)

In [ ]:
loans.groupby('loan_status')['fico_score'].describe()

#### Open credit lines and Total accounts

In [ ]:
loans[['open_acc','total_acc']].corr()

In [ ]:
loans.groupby('loan_status')['open_acc'].describe()

In [ ]:
loans['acc_ratio'] = loans['open_acc'] / loans['total_acc']

In [ ]:
loans.drop(['total_acc','open_acc'], axis=1, inplace=True)

#### Public records and Public records brankrupticies

In [ ]:
loans[['pub_rec','pub_rec_bankruptcies']].corr()

In [ ]:
loans.groupby('loan_status')['pub_rec'].describe()

In [ ]:
loans['pub_rec_bankruptcies'].value_counts().sort_index()

In [ ]:
plot_var('pub_rec_bankruptcies', 'Public Record Bankruptcies', continuous=False)

In [ ]:
loans.drop('pub_rec', axis=1, inplace=True)

#### Revolving balance

In [ ]:
loans['revol_bal'].describe()

In [ ]:
loans['log_revol_bal'] = loans['revol_bal'].apply(lambda x: np.log10(x+1))

In [ ]:
loans.drop('revol_bal', axis=1, inplace=True)

#### Revolving util

In [ ]:
loans['revol_util'].describe()

In [ ]:
loans[['revol_util','fico_score']].corr()

In [ ]:
loans.groupby('loan_status')['revol_util'].describe()

#### Mortgage account

In [ ]:
loans['mort_acc'].describe()

In [ ]:
loans.groupby('loan_status')['mort_acc'].describe()

### Date type columns

In [ ]:
loans['issue_d'].sample(5)

In [ ]:
loans['earliest_cr_line'].sample(5)

In [ ]:
loans['earliest_cr_line'].isnull().any()

In [ ]:
loans['issue_d'].isnull().any()

In [ ]:
loans['issue_d'] = pd.to_datetime(loans['issue_d'])

In [ ]:
plt.figure(figsize=(6,3), dpi=90)
loans['issue_d'].dt.year.value_counts().sort_index().plot.bar(color='darkblue')
plt.xlabel('Year')
plt.ylabel('Number of Loans Funded')
plt.title('Loans Funded per Year')

In [ ]:
loans['earliest_cr_line'] = pd.to_datetime(loans['earliest_cr_line'])

Calculate number of days between issue date and earliest credit lines, so that we know the days of applicant with the bank

In [ ]:
loans['days_from_issue_to_earliest_cr'] = (loans['issue_d'] - loans['earliest_cr_line']).apply(lambda x: x.days)

In [ ]:
loans['days_from_issue_to_earliest_cr'].sample(5)

In [ ]:
loans['target'] = (loans['loan_status'] == 'Charged Off').astype(int)
loans['target'].describe()

In [ ]:
_df = loans.groupby('days_from_issue_to_earliest_cr')['target'].mean().reset_index()

In [ ]:
sns.scatterplot(x='days_from_issue_to_earliest_cr', y='target', data=_df)

In [ ]:
_df.corr()

In [ ]:
loans.drop(['target','days_from_issue_to_earliest_cr','earliest_cr_line'],axis=1,inplace=True)

#### More Pre preprocessing

Convert charge off in loan status as 0/1 indicator

In [ ]:
loans['charged_off'] = (loans['loan_status'] == 'Charged Off').apply(np.uint8)
loans.drop('loan_status', axis=1, inplace=True)

In [ ]:

missing_fractions = loans.isnull().mean().sort_values(ascending=False) # Fraction of data missing for each variable

In [ ]:

print(missing_fractions[missing_fractions > 0]) # Print variables that are missing data

In [ ]:

print(loans.columns)

Create dummy variables for categorical data

In [ ]:
loans = pd.get_dummies(loans, columns=['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state', 'initial_list_status','application_type'], drop_first=True)



In [ ]:
loans.shape

In [ ]:
import pyarrow.feather as feather
feather.write_feather(loans,'./loans_modelling_data')